# 📊 Cumulative Process of Tickets - RAG Evaluation

**Version**: 1.0  
**Features**: Real data evaluation, complete RAG metrics, multi-model comparison  
**Output**: Compatible cumulative_results_xxxxx.json for Streamlit visualization  

**Complete Evaluation Pipeline:**
- Real embedding generation using actual models (Ada, E5-Large, MPNet, MiniLM)
- CrossEncoder reranking with Min-Max normalization
- Complete RAGAS metrics (faithfulness, answer_relevancy, answer_correctness, context_precision, context_recall, semantic_similarity)
- Complete BERTScore metrics (bert_precision, bert_recall, bert_f1)
- Proper URL normalization for ground truth matching

---

## 🚀 1. Setup

In [ ]:
!pip install torch

In [ ]:
!pip install ranx

In [ ]:
from importlib.metadata import version
print(version("ranx"))

In [ ]:
# Mount Google Drive and install packages
from google.colab import drive
drive.mount('/content/drive')

!pip install -q sentence-transformers pandas numpy scikit-learn openai python-dotenv tqdm torch bert-score

import sys
import os
import glob
import re
from datetime import datetime

# Setup paths
BASE_PATH = '/content/drive/MyDrive/TesisMagister/acumulative/colab_data/'
ACUMULATIVE_PATH = '/content/drive/MyDrive/TesisMagister/acumulative/'
RESULTS_OUTPUT_PATH = ACUMULATIVE_PATH

# Add to Python path
sys.path.append(BASE_PATH)

# Load API keys
try:
    from google.colab import userdata
    openai_key = userdata.get('OPENAI_API_KEY')
    if openai_key:
        os.environ['OPENAI_API_KEY'] = openai_key
        print("✅ OpenAI API key loaded")

    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        from huggingface_hub import login
        login(token=hf_token)
        print("✅ HF token loaded")
except:
    print("⚠️ API keys not found in secrets")

# Embedding files
EMBEDDING_FILES = {
    'ada': BASE_PATH + 'docs_ada_with_embeddings_20250721_123712.parquet',
    'e5-large': BASE_PATH + 'docs_e5large_with_embeddings_20250721_124918.parquet',
    'mpnet': BASE_PATH + 'docs_mpnet_with_embeddings_20250721_125254.parquet',
    'minilm': BASE_PATH + 'docs_minilm_with_embeddings_20250721_125846.parquet'
}

print("✅ Setup complete")

In [ ]:
# GPU Memory Configuration (AÑADIR COMO NUEVA CELDA)
import torch
import gc
import os

# Configurar PyTorch para mejor manejo de memoria
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Limitar uso de GPU al 80%
if torch.cuda.is_available():
    torch.cuda.set_per_process_memory_fraction(0.8)
    print(f"GPU disponible: {torch.cuda.get_device_name()}")
    print(f"Memoria total: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f} GB")

def clear_gpu_memory():
    torch.cuda.empty_cache()
    gc.collect()
    print("🧹 GPU memory cleared")

def check_gpu_memory():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated()/1024**3
        reserved = torch.cuda.memory_reserved()/1024**3
        print(f"🔍 GPU Memory - Allocated: {allocated:.2f}GB, Reserved: {reserved:.2f}GB")

print("✅ GPU configuration loaded")

## 📚 2. Load Evaluation Code

In [ ]:
# Complete evaluation code
import pandas as pd
import numpy as np
import json
import time
import pytz
import hashlib
from datetime import datetime
from pathlib import Path
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, CrossEncoder
from openai import OpenAI
from urllib.parse import urlparse, urlunparse
import warnings
warnings.filterwarnings('ignore')

### openai cache

In [ ]:
# ===========================
# OPENAI CACHE SYSTEM
# ===========================

class OpenAICache:
    """Manages caching of OpenAI API responses to minimize costs"""

    def __init__(self, cache_path: str):
        self.cache_path = cache_path
        self.cache = {"metadata": {}, "cached_responses": {}}
        self.hits = 0
        self.misses = 0
        self.modified = False
        self.load_cache()

    def _create_hash(self, question: str, context_links: list = None, prompt_type: str = "answer") -> str:
        """Create unique hash for question + context combination"""
        # Normalize context links
        context_str = ""
        if context_links:
            sorted_links = sorted([normalize_url(link) for link in context_links if link])
            context_str = "|".join(sorted_links)

        # Create hash string
        hash_input = f"{prompt_type}:{question}:{context_str}"
        return hashlib.md5(hash_input.encode('utf-8')).hexdigest()

    def load_cache(self):
        """Load existing cache from file"""
        try:
            if os.path.exists(self.cache_path):
                with open(self.cache_path, 'r', encoding='utf-8') as f:
                    self.cache = json.load(f)

                total = len(self.cache.get('cached_responses', {}))
                print(f"✅ Loaded OpenAI cache: {total} cached entries")
            else:
                print("📝 No existing cache found, creating new cache")
                self.cache = {
                    "metadata": {
                        "cache_version": "1.0",
                        "created_at": datetime.now().isoformat()
                    },
                    "cached_responses": {}
                }
        except Exception as e:
            print(f"⚠️ Error loading cache: {e}, starting fresh")
            self.cache = {"metadata": {}, "cached_responses": {}}

    def save_cache(self):
        """Save cache to file"""
        if not self.modified:
            return

        try:
            # Update metadata
            self.cache["metadata"]["total_entries"] = len(self.cache["cached_responses"])
            self.cache["metadata"]["last_updated"] = datetime.now().isoformat()
            self.cache["metadata"]["stats"] = {
                "hits": self.hits,
                "misses": self.misses,
                "hit_rate": self.hits / (self.hits + self.misses) if (self.hits + self.misses) > 0 else 0
            }

            with open(self.cache_path, 'w', encoding='utf-8') as f:
                json.dump(self.cache, f, ensure_ascii=False, indent=2)

            print(f"💾 Cache saved: {len(self.cache['cached_responses'])} entries")
            self.modified = False
        except Exception as e:
            print(f"❌ Error saving cache: {e}")

    def get(self, question: str, context_links: list = None, prompt_type: str = "answer"):
        """Get cached response if exists"""
        cache_key = self._create_hash(question, context_links, prompt_type)

        if cache_key in self.cache["cached_responses"]:
            self.hits += 1
            return self.cache["cached_responses"][cache_key]

        self.misses += 1
        return None

    def set(self, question: str, response_data: dict, context_links: list = None, prompt_type: str = "answer"):
        """Store response in cache"""
        cache_key = self._create_hash(question, context_links, prompt_type)

        self.cache["cached_responses"][cache_key] = {
            **response_data,
            "timestamp": datetime.now().isoformat()
        }
        self.modified = True

    def get_stats(self):
        """Get cache statistics"""
        total = self.hits + self.misses
        hit_rate = (self.hits / total * 100) if total > 0 else 0

        return {
            "total_queries": total,
            "cache_hits": self.hits,
            "cache_misses": self.misses,
            "hit_rate": hit_rate,
            "estimated_cost_saved": self.hits * 0.05  # Approx $0.05 per cached query
        }

# Global cache instance (will be initialized in evaluation)
openai_cache = None


### Classes

In [ ]:
# Global semantic similarity model (will be initialized in evaluation to avoid repeated loading)
semantic_similarity_model = None

def normalize_url(url: str) -> str:
    """
    Normalizes a URL by removing query parameters and fragments (anchors).

    Examples:
        https://learn.microsoft.com/en-us/azure/storage/blobs/storage-blob-overview?view=azure-cli-latest#overview
        -> https://learn.microsoft.com/en-us/azure/storage/blobs/storage-blob-overview

        https://learn.microsoft.com/azure/virtual-machines/windows/quick-create-portal?tabs=windows10#create-vm
        -> https://learn.microsoft.com/azure/virtual-machines/windows/quick-create-portal

    Args:
        url: The URL to normalize

    Returns:
        The normalized URL without query parameters and fragments
    """
    if not url or not url.strip():
        return ""

    try:
        # Parse the URL
        parsed = urlparse(url.strip())

        # Reconstruct without query parameters and fragments
        normalized = urlunparse((
            parsed.scheme,    # https
            parsed.netloc,    # learn.microsoft.com
            parsed.path,      # /en-us/azure/storage/blobs/storage-blob-overview
            '',               # params (empty)
            '',               # query (empty) - removes ?view=azure-cli-latest
            ''                # fragment (empty) - removes #overview
        ))

        return normalized
    except Exception as e:
        # If parsing fails, return the original URL stripped
        return url.strip()

class RealEmbeddingGenerator:
    """Generates real embeddings using actual models"""

    def __init__(self):
        self.models = {}
        self._load_models()

    def _load_models(self):
        """Load sentence transformer models"""
        model_configs = {
            'e5-large': 'intfloat/e5-large-v2',
            'mpnet': 'sentence-transformers/all-mpnet-base-v2',
            'minilm': 'sentence-transformers/all-MiniLM-L6-v2'
        }

        for name, model_path in model_configs.items():
            try:
                self.models[name] = SentenceTransformer(model_path)
                print(f"✅ Loaded {name} model")
            except Exception as e:
                print(f"❌ Error loading {name}: {e}")
                self.models[name] = None

    def validate_models(self):
        """Validate that all required models are loaded before evaluation"""
        required_models = ['e5-large', 'mpnet', 'minilm']
        failed_models = []

        for model_name in required_models:
            if model_name not in self.models or self.models[model_name] is None:
                failed_models.append(model_name)

        if failed_models:
            raise RuntimeError(
                f"Required models failed to load: {', '.join(failed_models)}. "
                f"Cannot proceed with evaluation. Please check model availability and try again."
            )

        print("✅ All sentence-transformer models validated and ready")

    def generate_query_embedding(self, question: str, model_name: str) -> np.ndarray:
        """Generate real query embedding for the given question"""

        if model_name == 'ada':
            # Use REAL OpenAI API for Ada embeddings
            try:
                client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
                response = client.embeddings.create(
                    input=question,
                    model="text-embedding-ada-002"
                )
                ada_embedding = np.array(response.data[0].embedding)
                return ada_embedding.astype(np.float32)
            except Exception as e:
                # CRITICAL: Fail explicitly instead of using random data
                raise RuntimeError(f"Failed to generate Ada embedding for question. Error: {e}")

        elif model_name in self.models and self.models[model_name]:
            try:
                # For sentence-transformer models, encode directly
                if model_name == 'mpnet':
                    # For MPNet, add query prefix as recommended
                    prefixed_question = f"query: {question}"
                    embedding = self.models[model_name].encode(prefixed_question)
                else:
                    embedding = self.models[model_name].encode(question)

                return embedding.astype(np.float32)
            except Exception as e:
                # CRITICAL: Fail explicitly instead of using random data
                raise RuntimeError(f"Failed to generate {model_name} embedding for question. Error: {e}")

        else:
            # CRITICAL: Fail explicitly for unknown or unloaded models
            raise ValueError(f"Model '{model_name}' is unknown or failed to load. Cannot generate embedding.")

class EmbeddedRetriever:
    """Handles document embedding retrieval and search"""

    def __init__(self, file_path: str, model_name: str):
        self.model_name = model_name
        self.file_path = file_path
        self.df = None
        self.embeddings = None
        self.embedding_dim = None
        self.load_data()

    def load_data(self):
        """Load embedding data from parquet file"""
        try:
            self.df = pd.read_parquet(self.file_path)

            # Get embeddings
            if 'embedding' in self.df.columns:
                embeddings_list = self.df['embedding'].tolist()
                self.embeddings = np.array(embeddings_list)
                self.embedding_dim = self.embeddings.shape[1] if len(self.embeddings) > 0 else 0
                print(f"✅ Loaded {len(self.df)} documents for {self.model_name} ({self.embedding_dim}D)")
            else:
                raise ValueError("No 'embedding' column found")

        except Exception as e:
            print(f"❌ Error loading {self.model_name}: {e}")
            self.df = pd.DataFrame()
            self.embeddings = np.array([])
            self.embedding_dim = 0

    def search(self, query_embedding: np.ndarray, top_k: int = 10):
        """Search for similar documents"""
        if len(self.embeddings) == 0:
            return []

        try:
            # Calculate cosine similarities
            similarities = cosine_similarity(query_embedding.reshape(1, -1), self.embeddings)[0]

            # Get top-k indices
            top_indices = np.argsort(similarities)[::-1][:top_k]

            results = []
            for idx in top_indices:
                if idx < len(self.df):
                    doc = self.df.iloc[idx]
                    results.append({
                        'rank': len(results) + 1,
                        'cosine_similarity': float(similarities[idx]),
                        'link': doc.get('link', ''),
                        'title': doc.get('title', ''),
                        'content': doc.get('content', '')
                    })

            # DETERMINISTIC: Sort by similarity desc, then by link asc (tie-breaking)
            results = sorted(
                results,
                key=lambda x: (-x['cosine_similarity'], x['link'])
            )

            # Re-assign ranks after deterministic sort
            for i, doc in enumerate(results):
                doc['rank'] = i + 1

            return results
        except Exception as e:
            print(f"⚠️ Search error for {self.model_name}: {e}")
            return []

class EmbeddedDataPipeline:
    """Main pipeline for embedded document retrieval and evaluation"""

    def __init__(self, base_path: str, embedding_files: dict):
        self.base_path = base_path
        self.embedding_files = embedding_files
        self.retrievers = {}
        self.real_embedding_generator = RealEmbeddingGenerator()
        self.cross_encoder = None
        self._load_retrievers()
        self._load_cross_encoder()

    def _load_retrievers(self):
        """Load all embedding retrievers"""
        for model_name, file_path in self.embedding_files.items():
            if os.path.exists(file_path):
                self.retrievers[model_name] = EmbeddedRetriever(file_path, model_name)
            else:
                print(f"❌ File not found for {model_name}: {file_path}")

    def _load_cross_encoder(self):
        """Load CrossEncoder for reranking"""
        try:
            self.cross_encoder = CrossEncoder('mixedbread-ai/mxbai-rerank-xsmall-v1')
            print("✅ CrossEncoder loaded")
        except Exception as e:
            print(f"❌ Error loading CrossEncoder: {e}")

    def load_config_file(self, config_path: str):
        """Load configuration file"""
        try:
            with open(config_path, 'r', encoding='utf-8') as f:
                return json.load(f)
        except Exception as e:
            print(f"❌ Error loading config: {e}")
            return None

    def get_system_info(self):
        """Get system information"""
        available_models = list(self.retrievers.keys())
        models_info = {}

        for model_name, retriever in self.retrievers.items():
            if retriever.df is not None and len(retriever.df) > 0:
                models_info[model_name] = {
                    'num_documents': len(retriever.df),
                    'embedding_dim': retriever.embedding_dim
                }
            else:
                models_info[model_name] = {'error': 'Failed to load'}

        return {
            'available_models': available_models,
            'models_info': models_info
        }

    def cleanup(self):
        """Clean up resources"""
        pass


### Methods

In [ ]:
def calculate_real_retrieval_metrics(ground_truth_links: list, retrieved_docs: list, top_k_values: list = None):
    """
    Calculate retrieval metrics using ranx for MAP, MRR, nDCG, Precision, Recall, and F1.

    This function is strict and relies exclusively on ranx. It will raise an
    error if ranx is not installed or if any other issue occurs during evaluation.
    No fallback to manual calculation is provided.
    """
    import warnings
    # 1. Strict dependency check
    try:
        from ranx import Qrels, Run, evaluate
    except ImportError:
        raise ImportError("ranx is not installed. Please install it with 'pip install ranx' to use this function.")
    '''
    from pprint import pprint
    print("\n[DEBUG] Documents entering the method:")
    for i, doc in enumerate(retrieved_docs, start=1):
      print(f"\nDocument #{i}:")
      safe_doc = doc.copy()
      if "content" in safe_doc and isinstance(safe_doc["content"], str):
          text = safe_doc["content"]
          safe_doc["content"] = text[:20] + "..." if len(text) > 20 else text
      pprint(safe_doc)

    print("\n[DEBUG] ground_truth_links entering the method:")
    for i, doc in enumerate(ground_truth_links, start=1):
      print(f"\ground_truth_links #{i}:")
      pprint(doc)
    '''

    if top_k_values is None:
        top_k_values = list(range(1, 51))

    # 2. Input validation
    if not ground_truth_links or not retrieved_docs:
        warnings.warn("Input validation failed: ground_truth_links or retrieved_docs is empty. Returning zeroed metrics.")
        metrics = {}
        for k in top_k_values:
            metrics.update({
                f"map@{k}": 0.0, f"mrr@{k}": 0.0, f"ndcg@{k}": 0.0,
                f"precision@{k}": 0.0, f"recall@{k}": 0.0, f"f1@{k}": 0.0
            })
        metrics.update({"map": 0.0, "mrr": 0.0, "ndcg": 0.0, "document_scores": []})
        return metrics

    # Normalize GT links and build Qrels
    normalized_gt = {normalize_url(link) for link in ground_truth_links if link}
    qrels_dict = {"q_1": {str(link): 1.0 for link in normalized_gt}}

    # 3. Enhanced score handling and Run building
    score_keys_priority = ("crossencoder_score", "cosine_similarity")
    chosen_key = None
    if retrieved_docs:
        for key in score_keys_priority:
            if key in retrieved_docs[0] and isinstance(retrieved_docs[0][key], (int, float)):
                chosen_key = key
                break

    run_scores = {}
    if chosen_key:
        for doc in retrieved_docs:
            link = normalize_url(doc.get("link", ""))
            if link:
                run_scores[link] = float(doc.get(chosen_key, 0.0))
    else:
        # Warn if falling back to positional ranking
        warnings.warn(f"No score key found from {score_keys_priority}. Falling back to positional ranking.")
        n = len(retrieved_docs)
        for i, doc in enumerate(retrieved_docs):
            link = normalize_url(doc.get("link", ""))
            if link:
                run_scores[link] = float(n - i) # Higher rank = higher score

    run_dict = {"q_1": run_scores}

    # 4. Strict ranx evaluation (no fallback)
    qrels = Qrels(qrels_dict)
    run = Run(run_dict)

    # Prepare metric list for ranx
    metrics_to_eval = []
    for k in top_k_values:
        metrics_to_eval.extend([
            f"map@{k}", f"mrr@{k}", f"ndcg@{k}",
            f"precision@{k}", f"recall@{k}", f"f1@{k}"
        ])
    metrics_to_eval.extend(["map", "mrr", "ndcg"])

    # This will raise an exception if it fails
    results = evaluate(qrels, run, metrics=metrics_to_eval)

    # Add document scores for detailed analysis
    doc_scores = []
    for doc in retrieved_docs:
        doc_link = normalize_url(doc.get('link', ''))
        is_relevant = 1 if doc_link in normalized_gt else 0
        doc_scores.append({
            'rank': doc.get('rank', 0),
            'score': run_scores.get(doc_link, 0.0),
            'link': doc.get('link', ''),
            'title': doc.get('title', ''),
            'is_relevant': bool(is_relevant)
        })
    results['document_scores'] = doc_scores

    return results

#### RAG

In [ ]:
def calculate_rag_metrics_real(question: str, context_docs: list, generated_answer: str, ground_truth: str):
    """Calculate comprehensive RAG metrics using real OpenAI API and BERTScore (with caching)"""

    global openai_cache, semantic_similarity_model

    # Get context links for cache key
    context_links = [doc.get('link', '') for doc in context_docs[:3]]

    # Create unique cache key including generated_answer and ground_truth
    cache_input = f"{question}|{generated_answer}|{ground_truth}"

    # Try to get from cache first
    if openai_cache:
        cached = openai_cache.get(cache_input, context_links, prompt_type="rag_metrics")
        if cached:
            # Return cached metrics (excluding timestamp and context_links)
            return {k: v for k, v in cached.items() if k not in ['timestamp', 'context_links']}

    # Not in cache, calculate metrics
    try:
        client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

        # Prepare context
        context_text = "\n".join([doc.get('content', '')[:3000] for doc in context_docs[:3]])

        # OPTIMIZED: Single API call for all RAGAS metrics (6 calls → 1 call, 83% cost reduction)
        combined_prompt = f"""Evaluate this RAG system output across 5 dimensions. Respond ONLY with a JSON object.

Question: {question}

Context: {context_text}

Generated Answer: {generated_answer}

Ground Truth Answer: {ground_truth if ground_truth else "Not provided"}

Rate each dimension on a 1-5 scale and respond in this EXACT JSON format:
{{
  "faithfulness": <1-5>,
  "answer_relevancy": <1-5>,
  "answer_correctness": <1-5>,
  "context_precision": <1-5>,
  "context_recall": <1-5>
}}

Dimension definitions:
- faithfulness: Does the answer contradict the context? (1=contradicts, 5=fully supported)
- answer_relevancy: Is the answer relevant to the question? (1=irrelevant, 5=perfect)
- answer_correctness: How correct is the answer vs ground truth? (1=wrong, 5=correct, 3=no ground truth)
- context_precision: How relevant is the context for answering? (1=irrelevant, 5=precise)
- context_recall: Does context have all info needed for ground truth? (1=missing most, 5=complete, 3=no ground truth)

Respond with ONLY the JSON object, no other text."""

        # Initialize scores
        faithfulness_score = None
        relevancy_score = None
        correctness_score = None
        context_precision_score = None
        context_recall_score = None

        try:
            ragas_response = client.chat.completions.create(
                model="gpt-3.5-turbo-0125",
                messages=[{"role": "user", "content": combined_prompt}],
                max_tokens=150,
                temperature=0,
                response_format={"type": "json_object"}  # Force JSON response
            )

            # Parse JSON response
            metrics_json = json.loads(ragas_response.choices[0].message.content)

            # Extract and normalize scores (1-5 scale to 0-1)
            faithfulness_score = float(metrics_json.get("faithfulness", 0)) / 5.0 if metrics_json.get("faithfulness") else None
            relevancy_score = float(metrics_json.get("answer_relevancy", 0)) / 5.0 if metrics_json.get("answer_relevancy") else None
            correctness_score = float(metrics_json.get("answer_correctness", 0)) / 5.0 if metrics_json.get("answer_correctness") else None
            context_precision_score = float(metrics_json.get("context_precision", 0)) / 5.0 if metrics_json.get("context_precision") else None
            context_recall_score = float(metrics_json.get("context_recall", 0)) / 5.0 if metrics_json.get("context_recall") else None

        except json.JSONDecodeError as e:
            print(f"⚠️ Failed to parse RAGAS JSON response: {e}")
            # Scores remain None
        except Exception as e:
            print(f"⚠️ Failed to calculate RAGAS metrics: {e}")
            # Scores remain None

        # 6. BERTScore metrics (precision, recall, f1) and Semantic Similarity
        bert_precision = None
        bert_recall = None
        bert_f1 = None
        semantic_similarity = None

        try:
            if ground_truth and generated_answer:
                # Calculate real BERTScore using bert-score library
                from bert_score import score as bert_score_fn

                P, R, F1 = bert_score_fn(
                    [generated_answer],
                    [ground_truth],
                    lang='en',
                    model_type='microsoft/deberta-base-mnli',
                    device='cuda' if torch.cuda.is_available() else 'cpu',
                    batch_size=1,
                    verbose=False
                )

                # Free up GPU memory to prevent error CUDA out of memory
                torch.cuda.empty_cache()

                bert_precision = float(P[0])
                bert_recall = float(R[0])
                bert_f1s = F1.tolist()

                # Eliminar variables para liberar memoria
                del P, R, F1

                # Calculate semantic similarity separately using sentence transformers
                # Use global model to avoid loading it 2067 times (GPU memory leak fix)
                if semantic_similarity_model is not None:
                    gt_embedding = semantic_similarity_model.encode(ground_truth)
                    answer_embedding = semantic_similarity_model.encode(generated_answer)

                    similarity = cosine_similarity(
                        gt_embedding.reshape(1, -1),
                        answer_embedding.reshape(1, -1)
                    )[0][0]
                    semantic_similarity = float(similarity)
                else:
                    # Fallback if model not initialized (shouldn't happen)
                    print("⚠️ Warning: semantic_similarity_model not initialized")
                    semantic_similarity = None

        except Exception as e:
            # Fallback in case of errors
            print(f"⚠️ Failed to calculate BERTScore/semantic similarity: {e}")
            bert_precision = None
            bert_recall = None
            bert_f1 = None
            semantic_similarity = None

        # Prepare result dict
        result = {
            # RAGAS metrics
            'faithfulness': faithfulness_score,
            'answer_relevancy': relevancy_score,  # Note: using 'answer_relevancy' (with y) as expected by Streamlit
            'answer_correctness': correctness_score,
            'context_precision': context_precision_score,
            'context_recall': context_recall_score,
            'semantic_similarity': semantic_similarity,

            # BERTScore metrics
            'bert_precision': bert_precision,
            'bert_recall': bert_recall,
            'bert_f1': bert_f1,

            # Additional fields
            'evaluation_method': 'Complete_RAGAS_OpenAI_BERTScore'
        }

        # Save to cache
        if openai_cache:
            openai_cache.set(
                cache_input,
                {**result, "context_links": context_links},
                context_links,
                prompt_type="rag_metrics"
            )

        return result

    except Exception as e:
        print(f"⚠️ CRITICAL error in RAG metrics calculation: {e}")
        return {
            # RAGAS metrics - all None on critical error
            'faithfulness': None,
            'answer_relevancy': None,
            'answer_correctness': None,
            'context_precision': None,
            'context_recall': None,
            'semantic_similarity': None,

            # BERTScore metrics - all None on critical error
            'bert_precision': None,
            'bert_recall': None,
            'bert_f1': None,

            # Additional fields
            'evaluation_method': 'Critical_Error_Fallback'
        }

def generate_rag_answer(question: str, context_docs: list):
    """Generate answer using OpenAI GPT and context documents (with caching)"""

    global openai_cache

    # Get context links for cache key
    context_links = [doc.get('link', '') for doc in context_docs[:3]]

    # Try to get from cache first
    if openai_cache:
        cached = openai_cache.get(question, context_links, prompt_type="rag_answer")
        if cached:
            return cached.get('generated_answer', '')

    # Not in cache, call OpenAI
    try:
        client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

        # Prepare context from top documents
        context_text = "\n\n".join([
            f"Document {i+1}: {doc.get('content', '')[:800]}"
            for i, doc in enumerate(context_docs[:3])
        ])

        prompt = f"""
        Based on the following context documents, answer the question accurately and concisely.

        Context:
        {context_text}

        Question: {question}

        Answer:
        """

        response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
            temperature=0  # DETERMINISTIC: Changed from 0.1 to 0 for reproducibility
        )

        generated_answer = response.choices[0].message.content.strip()

        # Save to cache
        if openai_cache:
            openai_cache.set(
                question,
                {"generated_answer": generated_answer, "context_links": context_links},
                context_links,
                prompt_type="rag_answer"
            )

        return generated_answer

    except Exception as e:
        print(f"⚠️ Error generating RAG answer: {e}")
        return f"Error generating answer: {str(e)}"

def get_optimal_batch_size(documents: list, max_content_length: int = 1200) -> int:
    """Calculate optimal batch size based on content length (MEJORA DE RENDIMIENTO)"""
    if not documents:
        return 4

    # Estimar longitud promedio del contenido (sample primeros 5 documentos)
    sample_size = min(5, len(documents))
    avg_content_length = np.mean([
        len(doc.get('content', '')[:max_content_length]) + len(doc.get('title', ''))
        for doc in documents[:sample_size]
    ])

    # Batch size inversamente proporcional a longitud de contenido
    if avg_content_length > 1200:
        return 2  # Documentos largos → batch pequeño para evitar OOM
    elif avg_content_length > 600:
        return 4  # Documentos medianos → batch estándar
    else:
        return 8  # Documentos cortos → batch grande para velocidad

def rerank_with_cross_encoder(question: str, documents: list, cross_encoder, top_k: int = 10):
    """
    Rerank documents using CrossEncoder with MEJORAS CRÍTICAS:
    - Usa título + contenido (no solo contenido)
    - Batch size adaptativo según longitud de documentos
    - Mejora esperada: +15-25% NDCG@10, +10-20% MRR
    """

    if not cross_encoder or not documents:
        return documents

    try:
        # MEJORA 1: Prepare query-document pairs WITH TITLE + CONTENT
        pairs = []
        for doc in documents:
            title = doc.get('title', '').strip()
            content = doc.get('content', '').strip()

            # Combinar título y contenido limitado
            if title:
                # Título + primeros 1200 caracteres de contenido
                combined_text = f"{title}: {content[:1200]}"
            else:
                # Solo contenido limitado si no hay título
                combined_text = content[:1500]

            pairs.append([question, combined_text])

        # MEJORA 2: Batch size adaptativo según longitud de documentos
        optimal_batch_size = get_optimal_batch_size(documents, max_content_length=1200)

        # Get CrossEncoder scores con batch size optimizado
        scores = cross_encoder.predict(pairs, batch_size=optimal_batch_size)

        # Apply Min-Max normalization to convert logits to [0, 1] range
        scores = np.array(scores)
        if len(scores) > 1 and scores.max() != scores.min():
            # Standard Min-Max normalization
            normalized_scores = (scores - scores.min()) / (scores.max() - scores.min())
        else:
            # Fallback for edge cases (all scores identical)
            normalized_scores = np.full_like(scores, 0.5)

        # Add scores to documents and sort
        for i, doc in enumerate(documents):
            doc['crossencoder_score'] = float(normalized_scores[i])

        # DETERMINISTIC: Sort by CrossEncoder score desc, then by link asc (tie-breaking)
        reranked = sorted(
            documents,
            key=lambda x: (-x['crossencoder_score'], x.get('link', ''))
        )

        # Update ranks
        for i, doc in enumerate(reranked):
            doc['rank'] = i + 1

        return reranked[:top_k]

    except Exception as e:
        print(f"⚠️ CrossEncoder reranking error: {e}")
        return documents

def evaluate_single_model_complete(model_name: str, data_pipeline: EmbeddedDataPipeline,
                                   questions_data: list, reranking_method: str = 'crossencoder',
                                   top_k: int = 10, generate_rag: bool = True):
    """Complete evaluation for a single model with real embeddings and metrics"""

    print(f"\n🔄 Evaluating {model_name}...")

    retriever = data_pipeline.retrievers.get(model_name)
    if not retriever or len(retriever.df) == 0:
        print(f"❌ No valid retriever for {model_name}")
        return None

    all_before_metrics = []
    all_after_metrics = []
    individual_rag_metrics = []

    # Score accumulators
    before_scores = []
    after_scores = []
    ce_scores = []
    total_docs_reranked = 0

    # added for debug
    positions_before_ce = {
        'ada': [],
        'mpnet': [],
        'minilm': [],
        'e5-large': []
    }

    positions_after_ce = {
        'ada': [],
        'mpnet': [],
        'minilm': [],
        'e5-large': []
    }


    for i, question_data in enumerate(tqdm(questions_data, desc=f"{model_name}")):
        question = question_data['question']
        ground_truth_links = question_data.get('validated_links', [])
        ground_truth_answer = question_data.get('accepted_answer', '')

        # Memory cleanup and cache checkpoint
        if i % 100 == 0 and i > 0:
          clear_gpu_memory()
          print(f"🧹 Memoria limpiada en pregunta {i}")

        # Save cache checkpoint every 50 questions
        if i % 50 == 0 and i > 0 and openai_cache:
            openai_cache.save_cache()
            stats = openai_cache.get_stats()
            print(f"💾 Cache checkpoint @ {i}: {stats['cache_hits']} hits, {stats['cache_misses']} misses, hit rate: {stats['hit_rate']:.1f}%")

        # Generate real query embedding
        query_embedding = data_pipeline.real_embedding_generator.generate_query_embedding(
            question, model_name
        )

        # Retrieve documents
        retrieved_docs = retriever.search(query_embedding, top_k=top_k)

        # debugresults
        if i == 0:  # Solo mostrar para la primera pregunta (puedes cambiar la condición)
            print(f"\n🔍 Pregunta: {question}")
            print(f"📄 Top-{top_k} documentos recuperados:")
            for doc in retrieved_docs:
                print(f"  Rank {doc['rank']}: {doc['title'][:80]}...")
                print(f"    URL: {doc['link']}")
                print(f"    Similarity: {doc['cosine_similarity']:.4f}")
                print()

        if not retrieved_docs:
            continue

        # Calculate BEFORE reranking metrics
        before_metrics = calculate_real_retrieval_metrics(
            ground_truth_links, retrieved_docs, list(range(1, top_k + 1))
        )

        # Calculate average cosine similarity (before)
        before_avg_score = np.mean([doc['cosine_similarity'] for doc in retrieved_docs])
        before_scores.append(before_avg_score)

        all_before_metrics.append(before_metrics)

        # AFTER reranking
        if reranking_method == 'crossencoder' and data_pipeline.cross_encoder:
            # Rerank with CrossEncoder
            reranked_docs = rerank_with_cross_encoder(
                question, retrieved_docs, data_pipeline.cross_encoder, top_k
            )

            # Calculate AFTER reranking metrics
            after_metrics = calculate_real_retrieval_metrics(
                ground_truth_links, reranked_docs, list(range(1, top_k + 1))
            )

            # Calculate CrossEncoder scores
            ce_question_scores = [doc.get('crossencoder_score', 0) for doc in reranked_docs]
            ce_avg_score = np.mean(ce_question_scores) if ce_question_scores else 0
            ce_scores.append(ce_avg_score)

            # After score (using original cosine similarities)
            after_avg_score = np.mean([doc['cosine_similarity'] for doc in reranked_docs])
            after_scores.append(after_avg_score)

            total_docs_reranked += len(reranked_docs)

            # Store CrossEncoder specific metrics
            after_metrics['model_crossencoder_scores'] = ce_question_scores
            after_metrics['model_avg_crossencoder_score'] = ce_avg_score
            after_metrics['model_total_documents_reranked'] = len(reranked_docs)

        else:
            # No reranking
            after_metrics = before_metrics.copy()
            reranked_docs = retrieved_docs
            after_scores.append(before_avg_score)

        all_after_metrics.append(after_metrics)

        # RAG Metrics (using reranked docs as context)
        if generate_rag:
            try:
                # Generate answer
                generated_answer = generate_rag_answer(question, reranked_docs[:3])

                # Calculate RAG metrics
                rag_metrics = calculate_rag_metrics_real(
                    question, reranked_docs[:3], generated_answer, ground_truth_answer
                )

                rag_metrics['question_index'] = i
                rag_metrics['generated_answer'] = generated_answer
                individual_rag_metrics.append(rag_metrics)

            except Exception as e:
                print(f"⚠️ RAG metrics error for question {i}: {e}")

    # Calculate averages
    def calculate_averages(metrics_list):
        if not metrics_list:
            return {}

        all_keys = set()
        for metrics in metrics_list:
            all_keys.update(metrics.keys())

        averages = {}
        for key in all_keys:
            if key != 'document_scores':  # Skip document scores in averages
                values = [m.get(key, 0) for m in metrics_list if isinstance(m.get(key), (int, float))]
                if values:
                    averages[key] = np.mean(values)

        return averages

    avg_before_metrics = calculate_averages(all_before_metrics)
    avg_after_metrics = calculate_averages(all_after_metrics)

    # Add model-level score metrics
    avg_before_metrics['model_avg_score'] = np.mean(before_scores) if before_scores else 0
    avg_after_metrics['model_avg_score'] = np.mean(after_scores) if after_scores else 0

    if reranking_method == 'crossencoder' and ce_scores:
        avg_after_metrics['model_avg_crossencoder_score'] = np.mean(ce_scores)
        avg_after_metrics['model_total_documents_reranked'] = total_docs_reranked

    # RAG averages - Complete RAGAS + BERTScore metrics
    # CRITICAL: Filter out None values before calculating averages
    rag_averages = {}
    if individual_rag_metrics:
        # Helper function to safely calculate average excluding None values
        def safe_mean(metric_name):
            values = [r[metric_name] for r in individual_rag_metrics if r[metric_name] is not None]
            return np.mean(values) if values else None

        rag_averages = {
            # RAGAS metrics averages (excluding None values)
            'avg_faithfulness': safe_mean('faithfulness'),
            'avg_answer_relevance': safe_mean('answer_relevancy'),  # Note: 'answer_relevancy' with y
            'avg_answer_correctness': safe_mean('answer_correctness'),
            'avg_context_precision': safe_mean('context_precision'),
            'avg_context_recall': safe_mean('context_recall'),
            'avg_semantic_similarity': safe_mean('semantic_similarity'),

            # BERTScore metrics averages (excluding None values)
            'avg_bert_precision': safe_mean('bert_precision'),
            'avg_bert_recall': safe_mean('bert_recall'),
            'avg_bert_f1': safe_mean('bert_f1'),

            # Status and count
            'rag_available': True,
            'total_rag_evaluations': len(individual_rag_metrics),
            'successful_calculations': {
                'faithfulness': len([r for r in individual_rag_metrics if r['faithfulness'] is not None]),
                'answer_relevancy': len([r for r in individual_rag_metrics if r['answer_relevancy'] is not None]),
                'answer_correctness': len([r for r in individual_rag_metrics if r['answer_correctness'] is not None]),
                'context_precision': len([r for r in individual_rag_metrics if r['context_precision'] is not None]),
                'context_recall': len([r for r in individual_rag_metrics if r['context_recall'] is not None]),
                'semantic_similarity': len([r for r in individual_rag_metrics if r['semantic_similarity'] is not None]),
                'bert_precision': len([r for r in individual_rag_metrics if r['bert_precision'] is not None]),
                'bert_recall': len([r for r in individual_rag_metrics if r['bert_recall'] is not None]),
                'bert_f1': len([r for r in individual_rag_metrics if r['bert_f1'] is not None])
            }
        }
    else:
        rag_averages = {'rag_available': False}

    return {
        'model_name': model_name,
        'full_model_name': model_name,
        'num_questions_evaluated': len(questions_data),
        'embedding_dimensions': retriever.embedding_dim,
        'total_documents': len(retriever.df),
        'avg_before_metrics': avg_before_metrics,
        'avg_after_metrics': avg_after_metrics,
        'all_before_metrics': all_before_metrics,
        'all_after_metrics': all_after_metrics,
        'rag_metrics': rag_averages,
        'individual_rag_metrics': individual_rag_metrics
    }

def run_real_complete_evaluation(available_models: list, config_data: dict,
                                 data_pipeline: EmbeddedDataPipeline,
                                 reranking_method: str = 'crossencoder',
                                 max_questions: int = None, debug: bool = False):
    """Run complete evaluation with real embeddings and metrics"""

    start_time = time.time()

    # Buscar questions_data primero, luego questions (compatibilidad)
    questions_data = config_data.get('questions_data', config_data.get('questions', []))
    if max_questions:
        # debugresults
        questions_data = questions_data[:2]
        questions_data = questions_data[:max_questions]

    params = config_data.get('params', {})
    # Buscar top_k primero en el nivel raíz, luego en params
    top_k = config_data.get('top_k', params.get('top_k', 10))
    # Buscar generate_rag_metrics en nivel raíz, luego en params
    generate_rag = config_data.get('generate_rag_metrics', params.get('generate_rag_metrics', True))

    # Validate that all models are loaded before starting evaluation
    print("🔍 Validating models...")
    data_pipeline.real_embedding_generator.validate_models()

    # Initialize OpenAI cache
    global openai_cache
    cache_path = os.path.join(data_pipeline.base_path, '..', 'openai_responses_cache.json')
    openai_cache = OpenAICache(cache_path)
    print(f"💾 OpenAI cache initialized: {cache_path}")

    # Initialize semantic similarity model ONCE (GPU memory leak fix)
    global semantic_similarity_model
    print("🔄 Loading semantic similarity model...")
    semantic_similarity_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    print("✅ Semantic similarity model loaded (will be reused across all 2067 questions)")

    print(f"🚀 Starting evaluation of {len(available_models)} models on {len(questions_data)} questions")
    print(f"📊 Reranking method: {reranking_method}")
    print(f"🎯 Top-K: {top_k}")
    print(f"🤖 RAG metrics: {generate_rag}")

    all_model_results = {}

    for model_name in available_models:
        result = evaluate_single_model_complete(
            model_name=model_name,
            data_pipeline=data_pipeline,
            questions_data=questions_data,
            reranking_method=reranking_method,
            top_k=top_k,
            generate_rag=generate_rag
        )

        if result:
            all_model_results[model_name] = result

            # Brief summary
            avg_f1 = result['avg_after_metrics'].get('f1@5', 0)
            avg_score = result['avg_after_metrics'].get('model_avg_score', 0)
            print(f"  ✅ {model_name}: F1@5={avg_f1:.3f}, Score={avg_score:.3f}")

    end_time = time.time()
    evaluation_duration = end_time - start_time

    # Save final cache and show statistics
    if openai_cache:
        openai_cache.save_cache()
        stats = openai_cache.get_stats()
        print(f"\n💾 OpenAI Cache Statistics:")
        print(f"  📊 Total queries: {stats['total_queries']}")
        print(f"  ✅ Cache hits: {stats['cache_hits']} ({stats['hit_rate']:.1f}%)")
        print(f"  ⚠️  Cache misses: {stats['cache_misses']}")
        print(f"  💰 Estimated cost saved: ${stats['estimated_cost_saved']:.2f}")

    evaluation_params = {
        'num_questions': len(questions_data),
        'models_evaluated': len(all_model_results),
        'reranking_method': reranking_method,
        'top_k': top_k,
        'generate_rag_metrics': generate_rag
    }

    return {
        'all_model_results': all_model_results,
        'evaluation_duration': evaluation_duration,
        'evaluation_params': evaluation_params
    }

def embedded_process_and_save_results(all_model_results: dict, output_path: str,
                                      evaluation_params: dict, evaluation_duration: float):
    """Process and save results in Streamlit-compatible format"""

    # Chile timezone
    chile_tz = pytz.timezone('America/Santiago')
    now_utc = datetime.now(pytz.UTC)
    now_chile = now_utc.astimezone(chile_tz)

    # Generate filename with date format YYYYMMDD_HHMMSS
    timestamp_str = now_chile.strftime('%Y%m%d_%H%M%S')
    filename = f"cumulative_results_{timestamp_str}.json"
    filepath = os.path.join(output_path, filename)

    # Create comprehensive results structure
    results_data = {
        'config': evaluation_params,
        'evaluation_info': {
            'timestamp': now_chile.isoformat(),
            'timezone': 'America/Santiago',
            'evaluation_type': 'cumulative_metrics_colab_multi_model',
            'total_duration_seconds': evaluation_duration,
            'models_evaluated': len(all_model_results),
            'questions_per_model': evaluation_params.get('num_questions', 0),
            'enhanced_display_compatible': True,
            'data_verification': {
                'is_real_data': True,
                'no_simulation': True,
                'no_random_values': True,
                'rag_framework': 'Complete_RAGAS_with_OpenAI_API',
                'reranking_method': f"{evaluation_params.get('reranking_method', 'none')}_reranking"
            }
        },
        'results': all_model_results
    }

    # Save to file
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, ensure_ascii=False, indent=2)

        print(f"✅ Results saved: {filename}")

        return {
            'json': filepath,
            'filename': filename,
            'chile_time': now_chile.strftime('%Y-%m-%d %H:%M:%S %Z')
        }

    except Exception as e:
        print(f"❌ Error saving results: {e}")
        return None

print("✅ Complete evaluation code loaded")


## ⚙️ 3. Load Configuration

In [ ]:
# Find latest config file
config_files = glob.glob(ACUMULATIVE_PATH + 'evaluation_config_*.json')

if config_files:
    files_with_timestamps = []
    for file in config_files:
        match = re.search(r'evaluation_config_(\d+)\.json', file)
        if match:
            timestamp = int(match.group(1))
            files_with_timestamps.append((timestamp, file))

    if files_with_timestamps:
        files_with_timestamps.sort(reverse=True)
        CONFIG_FILE_PATH = files_with_timestamps[0][1]
        latest_timestamp = files_with_timestamps[0][0]
        readable_time = datetime.fromtimestamp(latest_timestamp).strftime('%Y-%m-%d %H:%M:%S')
        print(f"✅ Latest config: {os.path.basename(CONFIG_FILE_PATH)} ({readable_time})")
    else:
        CONFIG_FILE_PATH = ACUMULATIVE_PATH + 'questions_with_links.json'
        print("⚠️ Using default questions file")
else:
    CONFIG_FILE_PATH = ACUMULATIVE_PATH + 'questions_with_links.json'
    print("⚠️ No config files found, using default")

# Initialize pipeline and load config
data_pipeline = EmbeddedDataPipeline(BASE_PATH, EMBEDDING_FILES)
config_data = data_pipeline.load_config_file(CONFIG_FILE_PATH)

if config_data and config_data.get('questions_data'):
    # Handle new config format with questions_data
    questions_data = config_data['questions_data']
    params = config_data.get('data_config', {})  # New format uses data_config instead of params

    # Convert new format to expected format for compatibility
    config_data['questions'] = questions_data
    config_data['params'] = {
        'top_k': params.get('top_k', 10),
        'reranking_method': params.get('reranking_method', 'crossencoder'),
        'generate_rag_metrics': True,  # Default for new format
        'use_llm_reranker': params.get('reranking_method', 'crossencoder') == 'crossencoder',
        'num_questions': params.get('num_questions', len(questions_data))
    }
    params = config_data['params']

    print(f"✅ New config format loaded: {len(questions_data)} questions")

elif config_data and config_data.get('questions'):
    # Handle legacy format
    params = config_data['params']
    print(f"✅ Legacy config format loaded: {len(config_data['questions'])} questions")

else:
    print("❌ Error loading config - no questions found")
    if config_data:
        print(f"Available keys: {list(config_data.keys())}")
    RERANKING_METHOD = 'crossencoder'
    params = {'top_k': 10, 'reranking_method': 'crossencoder', 'generate_rag_metrics': True}

if config_data and config_data.get('questions'):
    # Get reranking method with backward compatibility
    RERANKING_METHOD = params.get('reranking_method', 'crossencoder')
    USE_LLM_RERANKING = params.get('use_llm_reranker', True)

    if RERANKING_METHOD == 'crossencoder' and not USE_LLM_RERANKING:
        RERANKING_METHOD = 'none'

    print(f"🔄 Reranking method: {RERANKING_METHOD}")
    print(f"🎯 Top-K: {params.get('top_k', 10)}")
    print(f"📊 RAG metrics: {params.get('generate_rag_metrics', False)}")
else:
    print("❌ Error loading config")
    RERANKING_METHOD = 'crossencoder'

## 📊 4. Check Available Models

In [ ]:
# Get system info
system_info = data_pipeline.get_system_info()

print(f"📊 Available models:")
for model_name in system_info['available_models']:
    model_info = system_info['models_info'].get(model_name, {})
    if 'error' not in model_info:
        print(f"  ✅ {model_name}: {model_info.get('num_documents', 0):,} docs, {model_info.get('embedding_dim', 0)}D")
    else:
        print(f"  ❌ {model_name}: {model_info.get('error', 'Error')}")

available_models = [name for name in system_info['available_models']
                   if 'error' not in system_info['models_info'].get(name, {})]

# Added to reduce resources for testing
available_models = ['ada']

print(f"\n🎯 Models for evaluation: {available_models}")

## 🚀 5. Run Evaluation

In [ ]:
# CONFIGURACIONES MEJORADAS
BATCH_QUESTIONS = 15  # Reducido de 50 a 15

# Limpiar memoria antes de empezar
clear_gpu_memory()
check_gpu_memory()


In [ ]:
# Run evaluation
evaluation_result = run_real_complete_evaluation(
    available_models=available_models,
    config_data=config_data,
    data_pipeline=data_pipeline,
    reranking_method=RERANKING_METHOD,
    max_questions=None,  # Use all questions from config
    debug=False
)

all_models_results = evaluation_result['all_model_results']
evaluation_duration = evaluation_result['evaluation_duration']
evaluation_params = evaluation_result['evaluation_params']

print(f"\n✅ Evaluation completed in {evaluation_duration/60:.2f} minutes")

## 💾 6. Save Results

In [ ]:
# Save results
saved_files = embedded_process_and_save_results(
    all_model_results=all_models_results,
    output_path=RESULTS_OUTPUT_PATH,
    evaluation_params=evaluation_params,
    evaluation_duration=evaluation_duration
)

if saved_files:
    print(f"✅ Results saved:")
    print(f"  📄 File: {os.path.basename(saved_files['json'])}")
    print(f"  🌍 Time: {saved_files['chile_time']}")
    print(f"  ✅ Format: Streamlit compatible")
else:
    print("❌ Error saving results")

## 📈 7. Results Summary

In [ ]:
# Display results summary
# Make sure to run the previous cell (Save Results) first!
if saved_files and 'json' in saved_files:
    import json

    with open(saved_files['json'], 'r') as f:
        final_results = json.load(f)

    print("📊 RESULTS SUMMARY")
    print("="*50)

    if 'results' in final_results:
        results_data = final_results['results']

        for model_name, model_data in results_data.items():
            before_metrics = model_data.get('avg_before_metrics', {})
            after_metrics = model_data.get('avg_after_metrics', {})

            print(f"\n📊 {model_name.upper()}:")
            print(f"  📝 Questions: {model_data.get('num_questions_evaluated', 0)}")
            print(f"  📄 Documents: {model_data.get('total_documents', 0):,}")

            if before_metrics and after_metrics:
                # Performance metrics
                f1_before = before_metrics.get('f1@5', 0)
                f1_after = after_metrics.get('f1@5', 0)
                improvement = ((f1_after - f1_before) / f1_before * 100) if f1_before > 0 else 0

                print(f"  📈 F1@5: {f1_before:.3f} → {f1_after:.3f} ({improvement:+.1f}%)")
                print(f"  📈 MRR: {before_metrics.get('mrr', 0):.3f} → {after_metrics.get('mrr', 0):.3f}")

                # Score metrics
                score_before = before_metrics.get('model_avg_score', 0)
                score_after = after_metrics.get('model_avg_score', 0)

                print(f"  📊 Avg Score: {score_before:.3f} → {score_after:.3f}")

                if 'model_avg_crossencoder_score' in after_metrics:
                    ce_score = after_metrics.get('model_avg_crossencoder_score', 0)
                    print(f"  🧠 CrossEncoder Score: {ce_score:.3f}")
                    print(f"  📊 Documents Reranked: {after_metrics.get('model_total_documents_reranked', 0)}")

            # RAG metrics - FIX: Check for None values before formatting
            rag_metrics = model_data.get('rag_metrics', {})
            if rag_metrics.get('rag_available'):
                print(f"  🤖 RAG Metrics Available: ✅")

                # Display only available metrics (non-None)
                if 'avg_faithfulness' in rag_metrics and rag_metrics['avg_faithfulness'] is not None:
                    print(f"    📋 Faithfulness: {rag_metrics['avg_faithfulness']:.3f}")

                if 'avg_answer_relevance' in rag_metrics and rag_metrics['avg_answer_relevance'] is not None:
                    print(f"    🎯 Answer Relevance: {rag_metrics['avg_answer_relevance']:.3f}")

                if 'avg_answer_correctness' in rag_metrics and rag_metrics['avg_answer_correctness'] is not None:
                    print(f"    ✅ Answer Correctness: {rag_metrics['avg_answer_correctness']:.3f}")

                if 'avg_context_precision' in rag_metrics and rag_metrics['avg_context_precision'] is not None:
                    print(f"    🎯 Context Precision: {rag_metrics['avg_context_precision']:.3f}")

                if 'avg_context_recall' in rag_metrics and rag_metrics['avg_context_recall'] is not None:
                    print(f"    📥 Context Recall: {rag_metrics['avg_context_recall']:.3f}")

                if 'avg_semantic_similarity' in rag_metrics and rag_metrics['avg_semantic_similarity'] is not None:
                    print(f"    🔗 Semantic Similarity: {rag_metrics['avg_semantic_similarity']:.3f}")

                if 'avg_bert_precision' in rag_metrics and rag_metrics['avg_bert_precision'] is not None:
                    print(f"    📊 BERT Precision: {rag_metrics['avg_bert_precision']:.3f}")

                if 'avg_bert_recall' in rag_metrics and rag_metrics['avg_bert_recall'] is not None:
                    print(f"    📊 BERT Recall: {rag_metrics['avg_bert_recall']:.3f}")

                if 'avg_bert_f1' in rag_metrics and rag_metrics['avg_bert_f1'] is not None:
                    print(f"    🎯 BERT F1: {rag_metrics['avg_bert_f1']:.3f}")
            else:
                print(f"  🤖 RAG Metrics: ❌")

        # Overall comparison
        print(f"\n🏆 OVERALL:")
        best_f1 = ("", 0)
        best_score = ("", 0)

        for model_name, model_data in results_data.items():
            after_metrics = model_data.get('avg_after_metrics', {})
            f1 = after_metrics.get('f1@5', 0)
            score = after_metrics.get('model_avg_score', 0)

            if f1 > best_f1[1]:
                best_f1 = (model_name, f1)
            if score > best_score[1]:
                best_score = (model_name, score)

        print(f"  🥇 Best F1@5: {best_f1[0]} ({best_f1[1]:.3f})")
        print(f"  📊 Best Score: {best_score[0]} ({best_score[1]:.3f})")

        # Methodology info
        data_verification = final_results.get('evaluation_info', {}).get('data_verification', {})
        print(f"\n🔬 VERIFICATION:")
        print(f"  ✅ Real data: {data_verification.get('is_real_data', False)}")
        print(f"  📊 Framework: {data_verification.get('rag_framework', 'N/A')}")
        print(f"  🔄 Method: {data_verification.get('reranking_method', 'N/A')}")

print("\n🎉 EVALUATION COMPLETE!")

In [ ]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

## 🧹 8. Cleanup

In [ ]:
# Cleanup
data_pipeline.cleanup()
import gc
gc.collect()

print("🧹 Cleanup completed")
print("🎯 Results ready for Streamlit import")